In [1]:
import pandas as pd

df = pd.read_json("vndb-tags-2023-01-10.json")
df_test = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/96c0bd/sunburst-coffee-flavors-complete.csv')


In [2]:
def change_tag(row):
    if len(row["parents"]) == 0:
        row["parents"] = 0
    else:
        row["parents"] = row["parents"][0]
    return row
    

df = df.apply(change_tag, axis=1)
root = pd.DataFrame(columns=df.columns, data=[['', False, "tech", "", 0, True, "Tags", None, False, 0]])
df = pd.concat([df, root], axis=0)

In [3]:
import plotly.graph_objects as go


fig = go.Figure(go.Treemap(
    ids= df["id"],
    labels=df["name"],
    parents=df["parents"],
    values=df["vns"],
    root_color="lightgrey"
))

fig.update_layout(
    margin = dict(t=50, l=25, r=25, b=25)
)
# fig.show()
fig.write_html("index.html")